# Exercice Titanic , Explications Of Parts Of the CODE 

# Importing Libraries 

In [ ]:
import os
import re
import sys
import tarfile
import email
import email.policy

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
import urllib.request


# Explications Of : %matplotlib inline


In [ ]:
#Enables inline plotting: Forces matplotlib visualizations to render directly in Jupyter notebooks instead of opening separate windows.

#Embeds static images: Outputs plots as static PNG graphics within the notebook for easy viewing and sharing.

# Importing the functions we need from the libraries 

In [ ]:

#
from collections import Counter
from html import unescape
from scipy.ndimage import shift
from scipy.sparse import csr_matrix

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_curve, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_predict, cross_val_score, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC


# Importing the DATASET FROM THE LINK : "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/titanic/"

In [ ]:
#
TITANIC_PATH = os.path.join("datasets", "titanic")
DOWNLOAD_URL = "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/titanic/"

# Downlanding The "titanic" data set and verifying if "Train.csv" and "test.csv Exist" ? 

In [ ]:

#
def fetch_titanic_data(url=DOWNLOAD_URL, path=TITANIC_PATH):
    if not os.path.isdir(path):
        os.makedirs(path)
    for filename in ("train.csv", "test.csv"):
        filepath = os.path.join(path, filename)
        if not os.path.isfile(filepath):
            print("Downloading", filename)
            urllib.request.urlretrieve(url + filename, filepath)

# what (fetch_titanic_data() ) do : 

In [ ]:
#Creates dataset directory: Checks if the TITANIC_PATH folder exists, and creates it (including parent directories) if missing.

#Downloads missing files: For train.csv/test.csv, downloads from DOWNLOAD_URL only if not already present locally.

# Reading the DATA of the Dataset 

In [ ]:
def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)


# Loading the dataset " train.csv " and " Test.csv " into a dataframe 

In [ ]:
train_data = load_titanic_data("train.csv")
test_data = load_titanic_data("test.csv")



# Print the 5 first raws of the Dataset "Train"

In [ ]:
print(train_data.head())
print("---")
print()

# Indexing the Column of "PassengerID" in the Two Dataset 

In [ ]:
train_data = train_data.set_index("PassengerId")
test_data = test_data.set_index("PassengerId")

# Displaying the details of the Column (Not-null count , Type )

In [ ]:

print(train_data.info())
print("---")
print()

# Calculate the median of the "Sex" gender

In [ ]:
train_data[train_data["Sex"]=="female"]["Age"].median()

#

# Calculate some statistics of the data of the "Train.csv" Dataset


In [ ]:
print(train_data.describe())
print("---")
print()

# Calulate the Count of Survived,Pclass,sex,embarked

In [ ]:
print(train_data["Survived"].value_counts())
print("---")
print()

print(train_data["Pclass"].value_counts())
print("---")
print()

#
print(train_data["Sex"].value_counts())
print("---")
print()

#
print(train_data["Embarked"].value_counts())
print("---")
print()



# Scaling and Imputing Pipeline For numerical Values

In [ ]:
num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

# Encoding Categorical Values

In [ ]:
cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse_output=False)),
    ])


num_attribs = ["Age", "SibSp", "Parch", "Fare"]
cat_attribs = ["Pclass", "Sex", "Embarked"]

preprocess_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])


# Preprocesses and prints training features

In [ ]:
X_train = preprocess_pipeline.fit_transform(
    train_data[num_attribs + cat_attribs])
print(X_train)
print("---")
print()

# Extracting Target 'Label'

In [ ]:
y_train = train_data["Survived"]

# 2 Importing the Classification Model 'Random Forest' and fit with the Data

In [ ]:
forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_clf.fit(X_train, y_train)


# Preprocesses and predicts on test data

In [ ]:
X_test = preprocess_pipeline.transform(test_data[num_attribs + cat_attribs])
y_pred = forest_clf.predict(X_test)


# Printing The performances Of the Random Forest 

In [ ]:
forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
print(forest_scores.mean())
print("---")
print()

# Evaluates SVM classifier via cross-validation

In [ ]:
svm_clf = SVC(gamma="auto")
svm_scores = cross_val_score(svm_clf, X_train, y_train, cv=10)
print(svm_scores.mean())
print("---")
print()

# Visualizes model comparison

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot([1]*10, svm_scores, ".")
plt.plot([2]*10, forest_scores, ".")
plt.boxplot([svm_scores, forest_scores], tick_labels=["SVM", "Random Forest"])  # actualizado
plt.ylabel("Accuracy", fontsize=14)
plt.show()

# Creates age groups and analyzes survival rates

In [ ]:
train_data["AgeBucket"] = train_data["Age"] // 15 * 15
train_data[["AgeBucket", "Survived"]].groupby(['AgeBucket']).mean()
print(train_data[["AgeBucket", "Survived"]].groupby(['AgeBucket']).mean())
print("---")
print()

# Creates a new feature and analyzes survival rates

In [ ]:

train_data["RelativesOnboard"] = train_data["SibSp"] + train_data["Parch"]
train_data[["RelativesOnboard", "Survived"]].groupby(['RelativesOnboard']).mean()
print(train_data[["RelativesOnboard", "Survived"]].groupby(['RelativesOnboard']).mean())
print("---")
print()
